In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("/Users/bytedance/MLTrader-Research/")
import sys
sys.path.insert(0, ".")
import research.orderbook_strategies.utils.helper as helper
import research.orderbook_strategies.utils.factor_analysis as factor_analysis
import research.orderbook_strategies.utils.factors as factors
from research.orderbook_strategies.utils.product_info import product_info
import itertools
import pandas as pd
from collections import OrderedDict
import functools
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import dask
from dask import compute, delayed
warnings.filterwarnings('ignore')
import scipy
from scipy.stats import norm
import seaborn as sns
product_list = ["OPUSDT","BTCUSDT", "ETHUSDT", "APTUSDT", "1000PEPEUSDT"]

In [3]:
date_str, product, period_list = "2023-08-01", "OPUSDT", [512,]
data = helper.get_data(product, date_str)

In [21]:
trades_data = helper.get_trades_data(product, date_str)
assert trades_data is not None, f"trades data is None for {product} {date_str}"
metric_data = helper.get_metrics_data(product, date_str)
if metric_data is None:
    print("metric data is None for ", product, date_str)
date = datetime.strptime(date_str, "%Y-%m-%d")
yst_str = (date - timedelta(days=1)).strftime("%Y-%m-%d")
tmr_str = (date + timedelta(days=1)).strftime("%Y-%m-%d")
yst_trades_data = helper.get_trades_data(product, yst_str)
yst_metrics = helper.get_metrics_data(product, yst_str)
tmr_metrics = helper.get_metrics_data(product, tmr_str)
tmr_trades_data = helper.get_trades_data(product, tmr_str)

In [22]:
metric_data = pd.concat([yst_metrics, metric_data, tmr_metrics], axis=0)
trades_data = pd.concat([yst_trades_data, trades_data, tmr_trades_data], axis=0)

In [23]:
display(metric_data.head(3))
display(trades_data.head(3))

,symbol,sum_open_interest,sum_open_interest_value,count_toptrader_long_short_ratio,sum_toptrader_long_short_ratio,count_long_short_ratio,sum_taker_long_short_vol_ratio
create_time,,,,,,,
2023-07-31 00:00:00,OPUSDT,63595299.7,1.032279e+08,0.927361,1.489970,0.908262,1.369386
2023-07-31 00:05:00,OPUSDT,63667454.9,1.027529e+08,0.932186,1.487062,0.908781,0.767015
2023-07-31 00:10:00,OPUSDT,63605143.2,1.029068e+08,0.923946,1.486527,0.896313,1.206884


,agg_trade_id,price,quantity,first_trade_id,last_trade_id,is_buyer_maker
transact_time,,,,,,
2023-07-31 00:00:00.024,138804427,1.6232,370.8,340673883,340673885,False
2023-07-31 00:00:00.230,138804429,1.6232,1182.8,340673891,340673893,True
2023-07-31 00:00:01.592,138804430,1.6232,202.9,340673894,340673894,True


In [24]:
names = []
for name in ['sum_open_interest', 'sum_open_interest_value', 'count_long_short_ratio', 'sum_taker_long_short_vol_ratio']:
    metric_data[f"{name}_log_diff"] = np.log(metric_data[name]).diff()
    metric_data[f"{name}_log_diff"].iloc[0] = 0
    names.append(name)
    names.append(f"{name}_log_diff")
    
metric_data["more_position.12"] = (metric_data["sum_open_interest_value_log_diff"] > 0).rolling(12).mean()
names.append("more_position.12")
assert trades_data.index.isnull().sum() == 0, f"trades data has null index for {product} {date_str}"
assert metric_data.index.isnull().sum() == 0, f"metric_data has null index for {product} {date_str}"

In [19]:
merged_data = pd.merge_asof(trades_data, metric_data[names], left_index=True, right_index=True, direction='backward')

In [25]:
merged_data.head(3)

,agg_trade_id,price,quantity,first_trade_id,last_trade_id,is_buyer_maker,good,sum_open_interest,sum_open_interest_log_diff,sum_open_interest_value,sum_open_interest_value_log_diff,count_long_short_ratio,count_long_short_ratio_log_diff,sum_taker_long_short_vol_ratio,sum_taker_long_short_vol_ratio_log_diff,more_position.12
transact_time,,,,,,,,,,,,,,,,
2023-07-31 00:00:00.024,138804427,1.6232,370.8,340673883,340673885,False,NaN,63595299.7,0.0,1.032279e+08,0.0,0.908262,0.0,1.369386,0.0,NaN
2023-07-31 00:00:00.230,138804429,1.6232,1182.8,340673891,340673893,True,NaN,63595299.7,0.0,1.032279e+08,0.0,0.908262,0.0,1.369386,0.0,NaN
2023-07-31 00:00:01.592,138804430,1.6232,202.9,340673894,340673894,True,NaN,63595299.7,0.0,1.032279e+08,0.0,0.908262,0.0,1.369386,0.0,NaN


In [ ]:
merged_data['quote.qty'] = merged_data['quote.qty'].fillna(0)
for period in period_list:
    pass